## STRATEGY 

Parameters:
	Stretch:	0.5
	LongTermMALen:	200
	ShortTermMALen:	20
		
Data:
	ATRValue:	ATR(3)
	ADXValue:	ADX(5)
	
	ShortTermMA:	MA(Close, ShortTermMALen)
	LongTermMA:	MA(Close, LongTermMALen)

	ClosingRange:	(Close - Low) / (High - Low)
	IsUptrend: 	Close > LongTermMA
	IsVolatile:	ADXValue > 30			  

	Long_limit:	    Low - (ATRValue * Stretch) // limit order price
	Long_trigger:	ClosingRange < 0.3 // close in lower 30% of bar
	Long_setup:	    LongTrigger and IsUptrend and IsLiquid and IsVolatile
	
	StopLossExit:	Close < ShortTermMA



	Exit_rule:	Close > FillPrice or StopLossExit
	Exit_price:	NextOpen

1. go through and simplify or explain with comments
1b. try to use pairname to access data for better readability
2. use try to create object (not to long)
3. start process from beginning to see how repeadble it is 

In [57]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta
from numba import njit
import plotly.graph_objs as go


## data collection

In [58]:

symbols = ["ETH-USDT"]
timeframe = "1hour"
start_time = "2025-01-08 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")


# Fetch data for each symbol and store in a dictionary
dataframes = {}
for symbol in symbols:
    fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
    df = fetcher.fetch_candles_as_df()
    df['symbol'] = symbol
    dataframes[symbol] = df

# Print the DataFrame for each symbol
for symbol, df in dataframes.items():
    print(f"\nDataFrame for {symbol}:")
    print(df)
# fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)




INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 1, Candles: 552



DataFrame for ETH-USDT:
                              open    close     high      low         volume  \
timestamp                                                                      
2025-01-08 10:00:00+00:00  3347.14  3362.51  3373.15  3346.65   1480.4696085   
2025-01-08 11:00:00+00:00  3362.50  3349.35  3367.88  3323.31   2987.2982615   
2025-01-08 12:00:00+00:00  3349.35  3339.64  3360.95  3338.81   2466.9161027   
2025-01-08 13:00:00+00:00  3339.64  3358.53  3368.55  3339.64  3854.46036772   
2025-01-08 14:00:00+00:00  3358.53  3369.70  3381.46  3347.83   3661.1849986   
...                            ...      ...      ...      ...            ...   
2025-01-31 05:00:00+00:00  3216.57  3245.64  3248.67  3216.02    764.6370196   
2025-01-31 06:00:00+00:00  3245.64  3262.72  3271.60  3241.74   1093.4975792   
2025-01-31 07:00:00+00:00  3262.74  3241.78  3276.76  3239.45   1215.8891971   
2025-01-31 08:00:00+00:00  3241.79  3236.97  3248.54  3234.10    997.2620623   
2025-01-31 09:0

In [59]:
def custom_trading_strategy(close, high, low, stretch=0.5, 
                             long_term_ma_len=200, short_term_ma_len=20, adx_len=5, atr_len=3, symbol=None):
    # Calculate indicators
    atr = vbt.IndicatorFactory.from_talib('ATR').run(high, low, close, timeperiod=atr_len).real.to_numpy()
    adx = vbt.IndicatorFactory.from_talib('ADX').run(high, low, close, timeperiod=adx_len).real.to_numpy()
    
    # Moving Averages
    long_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=long_term_ma_len).real.to_numpy()
    short_term_ma = vbt.IndicatorFactory.from_talib('EMA').run(close, timeperiod=short_term_ma_len).real.to_numpy()

    # Closing Range
    closing_range = (close - low) / (high - low)
    
    # Conditions
    is_uptrend = close > long_term_ma
    is_volatile = adx > 30
    
    # Long Limit
    long_limit = low - (atr * stretch)
    
    # Long Trigger
    long_trigger = closing_range < 0.3
    
    # Long Setup
    long_setup = long_trigger & is_uptrend & is_volatile
    
    # Return limit order price when conditions for long setup are met
    limit_order_price = np.where(long_setup, long_limit, np.nan)
    
    # Ensure the order of outputs matches output_names
    return limit_order_price, long_term_ma,short_term_ma, atr, adx

# Create the indicator factory
custom_strategy_indicator = vbt.IndicatorFactory(
    class_name='CustomTradingStrategy',
    short_name='custom_strategy',
    input_names=['Close', 'High', 'Low'],
    param_names=['stretch', 'long_term_ma_len','short_term_ma_len', 'adx_len', 'atr_len','symbol'],
    output_names=['limit_order_price' ,'long_term_ma','short_term_ma', 'atr', 'adx']  # Add outputs for indicators

).from_apply_func(custom_trading_strategy)




In [60]:
# Entry parameter 
stretch = 0.5
long_term_ma_len = [100,200]
adx_len = [5,6]
atr_len = 3

# exit parameter
short_term_ma_len = [20,30,40]


indicator_obj = {}
for symbol in symbols:
    print(symbol)
    df = dataframes[symbol]

    # Usage example (commented out)
    indicator = custom_strategy_indicator.run(
        df['close'], df['high'], df['low'],
        stretch=stretch,
        long_term_ma_len=long_term_ma_len,
        short_term_ma_len=short_term_ma_len,
        adx_len=adx_len,
        atr_len=atr_len,
        symbol=symbol,
        param_product=True

    )

    indicator_obj[symbol] = indicator

# df_indicator_signals = indicator.limit_order_price
# df_indicator_signals




ETH-USDT


In [61]:
indicator_obj

{'ETH-USDT': <vectorbt.indicators.factory.CustomTradingStrategy at 0x12ceff190>}

In [62]:
def copy_rename_close_series(df_original, df_indicator_signals):

    def repeat_series_horizontally(series, target_shape):
        return np.tile(series.values.reshape(-1, 1), target_shape[1])

    # Assuming df and df_indicator_signals are already defined
    nedded_copies = len(df_indicator_signals.columns)

    # Repeat the 'close' series horizontally to match the shape of df_indicator_signals
    df_dublicate_close = pd.DataFrame(repeat_series_horizontally(df_original['close'], df_indicator_signals.shape), index=df_original.index)

    # Rename columns if shapes match
    if df_indicator_signals.shape == df_dublicate_close.shape:
        df_dublicate_close.columns = df_indicator_signals.columns
        print("Columns have been renamed")
    else:
        print("The DataFrames do not have the same shape.")
    
    
    return df_dublicate_close

# how to access the columns ----> e.g. df_dublicate_close[(0.5, 100, 20, 5, 3,'BTC-USDT')]
df_dublicate_close_pairs ={}
for symbol in symbols:
    df = dataframes[symbol]
    df_dublicate_close = copy_rename_close_series(df, indicator_obj[symbol].limit_order_price)
    df_dublicate_close_pairs[symbol] = df_dublicate_close
    # df_dublicate_close_pairs.append(df_dublicate_close)




Columns have been renamed


In [63]:
df_dublicate_close_pairs['ETH-USDT']

custom_strategy_stretch                0.5                             \
custom_strategy_long_term_ma_len       100                              
custom_strategy_short_term_ma_len       20                30            
custom_strategy_adx_len                  5        6        5        6   
custom_strategy_atr_len                  3        3        3        3   
custom_strategy_symbol            ETH-USDT ETH-USDT ETH-USDT ETH-USDT   
timestamp                                                               
2025-01-08 10:00:00+00:00          3362.51  3362.51  3362.51  3362.51   
2025-01-08 11:00:00+00:00          3349.35  3349.35  3349.35  3349.35   
2025-01-08 12:00:00+00:00          3339.64  3339.64  3339.64  3339.64   
2025-01-08 13:00:00+00:00          3358.53  3358.53  3358.53  3358.53   
2025-01-08 14:00:00+00:00          3369.70  3369.70  3369.70  3369.70   
...                                    ...      ...      ...      ...   
2025-01-31 05:00:00+00:00          3245.64  3245.64  3245.64  3245.64   
2025-01-31 06:00:00+00:00          3262.72  3262.72  3262.72  3262.72   
2025-01-31 07:00:00+00:00          3241.78  3241.78  3241.78  3241.78   
2025-01-31 08:00:00+00:00          3236.97  3236.97  3236.97  3236.97   
2025-01-31 09:00:00+00:00          3237.35  3237.35  3237.35  3237.35   

custom_strategy_stretch                                                \
custom_strategy_long_term_ma_len                         200            
custom_strategy_short_term_ma_len       40                20            
custom_strategy_adx_len                  5        6        5        6   
custom_strategy_atr_len                  3        3        3        3   
custom_strategy_symbol            ETH-USDT ETH-USDT ETH-USDT ETH-USDT   
timestamp                                                               
2025-01-08 10:00:00+00:00          3362.51  3362.51  3362.51  3362.51   
2025-01-08 11:00:00+00:00          3349.35  3349.35  3349.35  3349.35   
2025-01-08 12:00:00+00:00          3339.64  3339.64  3339.64  3339.64   
2025-01-08 13:00:00+00:00          3358.53  3358.53  3358.53  3358.53   
2025-01-08 14:00:00+00:00          3369.70  3369.70  3369.70  3369.70   
...                                    ...      ...      ...      ...   
2025-01-31 05:00:00+00:00          3245.64  3245.64  3245.64  3245.64   
2025-01-31 06:00:00+00:00          3262.72  3262.72  3262.72  3262.72   
2025-01-31 07:00:00+00:00          3241.78  3241.78  3241.78  3241.78   
2025-01-31 08:00:00+00:00          3236.97  3236.97  3236.97  3236.97   
2025-01-31 09:00:00+00:00          3237.35  3237.35  3237.35  3237.35   

custom_strategy_stretch                                                
custom_strategy_long_term_ma_len                                       
custom_strategy_short_term_ma_len       30                40           
custom_strategy_adx_len                  5        6        5        6  
custom_strategy_atr_len                  3        3        3        3  
custom_strategy_symbol            ETH-USDT ETH-USDT ETH-USDT ETH-USDT  
timestamp                                                              
2025-01-08 10:00:00+00:00          3362.51  3362.51  3362.51  3362.51  
2025-01-08 11:00:00+00:00          3349.35  3349.35  3349.35  3349.35  
2025-01-08 12:00:00+00:00          3339.64  3339.64  3339.64  3339.64  
2025-01-08 13:00:00+00:00          3358.53  3358.53  3358.53  3358.53  
2025-01-08 14:00:00+00:00          3369.70  3369.70  3369.70  3369.70  
...                                    ...      ...      ...      ...  
2025-01-31 05:00:00+00:00          3245.64  3245.64  3245.64  3245.64  
2025-01-31 06:00:00+00:00          3262.72  3262.72  3262.72  3262.72  
2025-01-31 07:00:00+00:00          3241.78  3241.78  3241.78  3241.78  
2025-01-31 08:00:00+00:00          3236.97  3236.97  3236.97  3236.97  
2025-01-31 09:00:00+00:00          3237.35  3237.35  3237.35  3237.35  

[552 rows x 12 columns]

In [73]:
import numpy as np
import vectorbt as vbt
from numba import njit
import talib
from vectorbt.portfolio.enums import SizeType, Direction

# Create arrays to store the data we want to plot
@njit
def order_func_nb(c, high, low, open_, entries, ma_short, entry_price):  # Added entry_price parameter
    close_price = c.close[c.i, c.col]
    close_minus_1bar = c.close[c.i-1, c.col]
    
    # if in position 
    if c.position_now > 0:
        if (close_minus_1bar <= ma_short[c.i-1,c.col]) or (close_price > entry_price[c.i]):
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,
                price=open_[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # Store exit data
            return value

    # if not in position search for position to enter
    elif (c.position_now == 0) and (c.i != 0):
        if (entries[c.i-1,c.col] > 0) and (low[c.i] < entries[c.i-1,c.col]):
            entry_price[:] = np.nan  # Reset entry price array
            entry_price[:] = entries[c.i-1,c.col]  # Update entry price array

            order = vbt.portfolio.nb.order_nb(
                size=1,
                price=entry_price[c.i],
                size_type=SizeType.Percent,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002,
                allow_partial=False,
                raise_reject=True
            )
            return order

    return vbt.portfolio.enums.NoOrder



In [74]:
pf_pairs = {}

for symbol in symbols:
    # candle data
    close = df_dublicate_close_pairs[symbol]
    open_ =dataframes[symbol]['open'].to_numpy().flatten()
    high = dataframes[symbol]['high'].to_numpy().flatten()
    low = dataframes[symbol]['low'].to_numpy().flatten()
    # high = df['high'].to_numpy().flatten()
    # low = df['low'].to_numpy().flatten()

    # indicator data
    entries = indicator_obj[symbol].limit_order_price.to_numpy()
    # entries = df_indicator_signals.to_numpy()
    ma_short = indicator_obj[symbol].short_term_ma.to_numpy()
    # ma_short = indicator.short_term_ma.to_numpy().flatten()

    # Create arrays to store data
    entry_price = np.full(close.shape[0], np.nan)  # Initialize entry price array

    # Create portfolio with trade_data
    pf = vbt.Portfolio.from_order_func(
        close,
        order_func_nb,
        high,
        low,
        open_,
        entries,
        ma_short,
        entry_price,  # Pass entry_price array to the function
        init_cash=500
    )

    pf_pairs[symbol] = pf



In [75]:
test= pf_pairs['ETH-USDT'][(0.5, 100,  20, 5,3,'ETH-USDT')]
pf_pairs


{'ETH-USDT': <vectorbt.portfolio.base.Portfolio at 0x12c32c0d0>}

In [95]:
# Combine the total return and max drawdown for each symbol
return_and_maxdd_list = []

for symbol in symbols:
    total_return = pf_pairs[symbol].total_return()
    max_dd = pf_pairs[symbol].max_drawdown()
    
    # Calculate the number of long, short, and total trades
    num_long_trades = pf_pairs[symbol].trades.records_readable.query('Direction == "Long"').shape[0]
    num_short_trades = pf_pairs[symbol].trades.records_readable.query('Direction == "Short"').shape[0]
    num_total_trades = num_long_trades + num_short_trades
    sharpe_ratio = pf_pairs[symbol].sharpe_ratio()
    sortino_ratio = pf_pairs[symbol].sortino_ratio()
    beta = pf_pairs[symbol].beta()
    
    # Create a DataFrame with the calculated values
    return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
    return_and_maxdd['longs'] = num_long_trades
    return_and_maxdd['shorts'] = num_short_trades
    return_and_maxdd['total_trades'] = num_total_trades
    return_and_maxdd['sharpe_ratio'] = sharpe_ratio
    return_and_maxdd['sortino_ratio'] = sortino_ratio
    return_and_maxdd['beta'] = beta
    return_and_maxdd_list.append(return_and_maxdd)

# Concatenate all DataFrames along the rows
combined_df = pd.concat(return_and_maxdd_list, axis=0)

combined_df

AttributeError: 'Portfolio' object has no attribute 'win_rate'

In [96]:
pf_pairs['ETH-USDT'][(0.5, 100,  20, 5,3,'ETH-USDT')].stats()

Start                         2025-01-08 10:00:00+00:00
End                           2025-01-31 09:00:00+00:00
Period                                 23 days 00:00:00
Start Value                                       500.0
End Value                                    444.895265
Total Return [%]                             -11.020947
Benchmark Return [%]                          -3.722219
Max Gross Exposure [%]                            100.0
Total Fees Paid                               10.465642
Max Drawdown [%]                              11.020947
Max Drawdown Duration                  16 days 22:00:00
Total Trades                                         11
Total Closed Trades                                  11
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  18.181818
Best Trade [%]                                 0.264241
Worst Trade [%]                               -3

In [13]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
total_return
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(50)

total_return  \
custom_strategy_stretch custom_strategy_long_term_ma_len custom_strategy_short_term_ma_len custom_strategy_adx_len custom_strategy_atr_len custom_strategy_symbol                 
0.5                     100                              20                                5                       3                       SOL-USDT                   -0.070556   
                                                         30                                5                       3                       SOL-USDT                   -0.070556   
                                                         40                                5                       3                       SOL-USDT                   -0.070556   
                        200                              20                                5                       3                       SOL-USDT                   -0.132418   
                                                         30                                5                       3                       SOL-USDT                   -0.132418   
                                                         40                                5                       3                       SOL-USDT                   -0.132418   
                        100                              20                                6                       3                       SOL-USDT                   -0.147310   
                                                         30                                6                       3                       SOL-USDT                   -0.147310   
                                                         40                                6                       3                       SOL-USDT                   -0.147310   
                        200                              20                                6                       3                       SOL-USDT                   -0.149988   
                                                         30                                6                       3                       SOL-USDT                   -0.149988   
                                                         40                                6                       3                       SOL-USDT                   -0.149988   

                                                                                                                                                                   max_drawdown  
custom_strategy_stretch custom_strategy_long_term_ma_len custom_strategy_short_term_ma_len custom_strategy_adx_len custom_strategy_atr_len custom_strategy_symbol                
0.5                     100                              20                                5                       3                       SOL-USDT                   -0.088359  
                                                         30                                5                       3                       SOL-USDT                   -0.088359  
                                                         40                                5                       3                       SOL-USDT                   -0.088359  
                        200                              20                                5                       3                       SOL-USDT                   -0.181901  
                                                         30                                5                       3                       SOL-USDT                   -0.181901  
                                                         40                                5                       3                       SOL-USDT                   -0.181901  
                        100                              20                                6                       3                       SOL-USDT                   -0.195153  
                                                      

In [14]:
trades = pf.trades.records_readable
trades
# combined_df = df.join(trades.set_index('Entry Timestamp'), how='outer')  # Join on index
# combined_df


,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,"(0.5, 100, 20, 5, 3, SOL-USDT)",2.684081,2025-01-14 12:00:00+00:00,186.097414,0.499500,2025-01-14 13:00:00+00:00,185.693868,0.498417,-2.081067,-0.004166,Long,Closed,0
1,1,"(0.5, 100, 20, 5, 3, SOL-USDT)",2.634254,2025-01-15 09:00:00+00:00,188.828217,0.497422,2025-01-15 13:00:00+00:00,186.009236,0.489996,-8.413329,-0.016914,Long,Closed,1
2,2,"(0.5, 100, 20, 5, 3, SOL-USDT)",2.374501,2025-01-16 14:00:00+00:00,205.945025,0.489017,2025-01-16 15:00:00+00:00,207.378412,0.492420,2.422142,0.004953,Long,Closed,2
3,3,"(0.5, 100, 20, 5, 3, SOL-USDT)",2.245417,2025-01-17 14:00:00+00:00,218.861911,0.491436,2025-01-17 15:00:00+00:00,218.856410,0.491424,-0.995211,-0.002025,Long,Closed,3
4,4,"(0.5, 100, 20, 5, 3, SOL-USDT)",1.966194,2025-01-18 20:00:00+00:00,249.437306,0.490442,2025-01-18 21:00:00+00:00,248.071862,0.487757,-3.662928,-0.007469,Long,Closed,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,97,"(0.5, 200, 40, 6, 3, SOL-USDT)",1.983809,2025-01-18 20:00:00+00:00,249.437306,0.494836,2025-01-18 21:00:00+00:00,248.071862,0.492127,-3.695745,-0.007469,Long,Closed,97
98,98,"(0.5, 200, 40, 6, 3, SOL-USDT)",1.761540,2025-01-19 12:00:00+00:00,278.815091,0.491144,2025-01-19 13:00:00+00:00,276.848194,0.487679,-4.443591,-0.009047,Long,Closed,98
99,99,"(0.5, 200, 40, 6, 3, SOL-USDT)",2.044841,2025-01-21 00:00:00+00:00,238.016014,0.486705,2025-01-21 02:00:00+00:00,233.905252,0.478299,-9.370858,-0.019254,Long,Closed,99
100,100,"(0.5, 200, 40, 6, 3, SOL-USDT)",1.855374,2025-01-22 23:00:00+00:00,257.276034,0.477343,2025-01-24 05:00:00+00:00,254.783412,0.472719,-5.574808,-0.011679,Long,Closed,100
